# Compressione JPEG

## Istruzioni:

- Eseguire la cella di codice.
- Verrà visualizzata sotto alla cella di codice l'interfaccia grafica:
  - Cliccando sul pulsante "Upload" si può caricare un'immagine.
  - Cliccando sul menù a tendina si può scegliere quale immagine tra quelle caricate si vuole comprimere.
  - Con i due slider F e d (aggiornati in modo dinamico in modo da non permettere di inserire valori "out of range") si possono scegliere i due parametri per la compressione.
  - Cliccando sul bottone "Comprimi!" viene chiamato l'algoritmo di compressione, al termine del quale verranno mostrate l'immagine originale e l'immagine compressa. Inoltre l'immagine compressa verrà salvata nella cartella "compressed_img"
  - Cliccando sul bottone "Crea archivio" viene generato un file .zip contenente tutte le immagini compresse, per poterle scaricare facilmente

In [20]:
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import rcParams

import numpy as np
from scipy.fft import dct, dctn, idctn
import math

import os
import shutil

from ipywidgets import interact_manual, widgets, IntSlider

### Funzione principale per comprimere immagine data dimensione blocchetto F e soglia d
def my_dct_compression(image, F, d):
  h = image.shape[0]
  w = image.shape[1]

  #andiamo a tagliare i pixel in eccesso rispetto al multiplo di F
  if h%F != 0:
    h = int(h/F) * F
  if w%F != 0:
    w = int(w/F) * F
  image_to_compress = image[0:h, 0:w]

  #nuovo array per l'immagine compressa
  compressed_image = np.zeros((h, w))

  # cicliamo sull'immagine a step di F
  for x in range(0,h,F):
    for y in range(0,w,F):
      cell = image_to_compress[x:x+F, y:y+F]   # ampiezza e larghezza della cella = F
      cell = dctn(cell, type = 2, norm = 'ortho') # DCT
      # cancelliamo le frequenze sotto la diagonale in base a d
      for i in range(0,F):
        for j in range(0,F):
          if i+j >= d:
            cell[i,j] = 0
      
      # calcoliamo l'inversa della DCT del blocchetto
      cell = idctn(cell, type = 2, norm = 'ortho')
      #arrotondiamo all'intero più vicino, 0 per i valori negativi e 255 per i valori più grandi di 255
      for i in range(0,F):
        for j in range(0,F):
          value = np.round(cell[i,j])
          if value < 0:
            value = 0
          elif value > 255:
            value = 255
          cell[i,j] = value
      compressed_image[x:x+F, y:y+F] = cell

  return compressed_image


### Creazione cartelle necessarie e lettura file
os.makedirs('img', exist_ok=True)
os.makedirs('compressed_img', exist_ok=True)

file=os.listdir('img/')
for item in file:
  if not item.endswith(".bmp"):
    file.remove(item)

### Magia dei widget per la UI

## Widget per upload immagini
uploader = widgets.FileUpload(accept='.bmp', multiple=False)

def _handle_upload(change):
    uploaded_filename = next(iter(uploader.value))
    content = uploader.value[uploaded_filename]['content']
    with open('img/'+uploaded_filename, 'wb') as f: f.write(content)
    file=os.listdir('img/')
    for item in file:
      if not item.endswith(".bmp"):
        file.remove(item)
    img_widget.options = file

uploader.observe(_handle_upload, names='data')

## Widget per menù a tendina
img_widget = widgets.Dropdown(
    options=file,
    description='Image:',
    disabled=False)

def update_F_range(*args):
  img = mpimg.imread(fname='img/'+img_widget.value, format='bmp')
  h = img.shape[0]
  w = img.shape[1]
  dim = min(h, w)
  F_widget.max = dim

img_widget.observe(update_F_range, 'value')

## Widget per slider F
F_widget = IntSlider(min=2, max=10, step=1)

def update_d_range(*args):
    d_widget.max = 2.0 * F_widget.value - 2

F_widget.observe(update_d_range, 'value')

## Widget per slider d
d_widget = IntSlider(min=1, max=10, step=1)


## Funzione che comprime immagine e fa plot
def call_dct_and_print(img, F, d):
  img_to_compress = mpimg.imread(fname='img/'+ img, format='bmp')
  # compressione immagine
  compressed_img = my_dct_compression(img_to_compress, F, d)

  # save compressed image
  img_filename = 'compressed_img/'+ img + '_F=' + str(F) + '_d=' + str(d) + '.png'
  plt.imsave(img_filename, arr=compressed_img, cmap='gray')

  # plot both images
  rcParams['figure.figsize'] = 25, 25
  print('\n')
  fig, ax = plt.subplots(1,2)
  ax[0].set_title('\n Original \n', fontsize=26)
  ax[1].set_title('\n F = ' + str(F) + ' d = ' + str(d) + '\n', fontsize=26)
  fig.patch.set_facecolor('#f7f7f7')
  ax[0].axis('off')
  ax[0].imshow(img_to_compress, cmap='gray')
  ax[1].axis('off')
  ax[1].imshow(compressed_img, cmap='gray')

## Funzione per creare uno zip con tutte le immagini compresse
def zip_compressed():
  shutil.make_archive('./compressed', 'zip', './compressed_img')

## Stampiamo la GUI
display(uploader)
UI = interact_manual(call_dct_and_print, img=img_widget, F=F_widget, d=d_widget)
UI.widget.children[3].description = 'Comprimi!'
UI_zip = interact_manual(zip_compressed)
UI_zip.widget.children[0].description = 'Crea archivio'


FileUpload(value={}, accept='.bmp', description='Upload')

interactive(children=(Dropdown(description='Image:', options=('flower_foveon.bmp', 'jump1.bmp', '160x160.bmp',…

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widge…